In [1]:
''' Import dependencies.

Moduels:
    tensorflow: The neural network framework.
    layer: The customized single layers.
    module: The customized multi-layer modules.
    
    os: The basic module for path parsing.
    json: The basic modue for config parsing.
    zipfile: Extract zip file.
    random: Shuffle indices.
    scipy.io: Load .mat file.
    numpy: To process .mat file data.
    
    tqdm: The 3rd-party looping visualzer.
'''

import layer, module
import tensorflow as tf
import os, json, zipfile, random, datetime, glob
import scipy.io, scipy.misc, imageio, math, numpy as np
from enum import Enum
from tqdm import tqdm, trange
from PIL import Image

/home/nulledge/Environments/tf-36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
''' Parsing config.

The config file is saved to root/config.
'''
CONFIG_PATH = os.path.abspath('../config.json')
with open(CONFIG_PATH) as CONFIG_FILE:
    CONFIG = json.loads(CONFIG_FILE.read())
    flags = tf.app.flags
    flags.DEFINE_string('project', CONFIG['project'], 'The project name.')
    
    flags.DEFINE_string('path_to_pretrained',
                        os.path.join(
                            os.path.expanduser(CONFIG['pretrained']['path']),
                            CONFIG['project']),
                        'The path to pretrained parameters.')
    flags.DEFINE_boolean('load_pretrained', CONFIG['pretrained']['load'], 'Load the latest pretrained parameters.')
    flags.DEFINE_boolean('save_pretrained', CONFIG['pretrained']['save'], 'Save the trained parameters.')
    
    flags.DEFINE_string('path_to_log',
                        os.path.join(
                            os.path.expanduser(CONFIG['log']['path']),
                            CONFIG['project']),
                        'The path to log.')
    flags.DEFINE_boolean('save_log', CONFIG['log']['save'], 'Save the log.')
    
    flags.DEFINE_string('path_to_data',
                        os.path.expanduser(CONFIG['data']['path']),
                        'The path to data.')
    flags.DEFINE_string('name_of_data', CONFIG['data']['name'], 'The name of data.')
    
    flags.DEFINE_string('task', CONFIG['task']['step'], 'The task to be done.')
    flags.DEFINE_integer('epoch', CONFIG['task']['train']['epoch'], 'The epoch to be trained.')
    flags.DEFINE_string('metric', CONFIG['task']['eval']['metric'], 'The evaluation metric.')
    flags.DEFINE_float('metric_coefficient', CONFIG['task']['eval']['coefficient'], 'The evaluation metric coefficient.')
    
FLAGS = flags.FLAGS

In [6]:
with tf.variable_scope('input'):
    images = tf.placeholder(
        name = 'image',
        dtype = tf.float32,
        shape = [None, 256, 256, 3])
    heatmaps_groundtruth = tf.placeholder(
        name = 'heatmap_groundtruth',
        dtype = tf.float32,
        shape = [None, 64, 64, len(Joint)])
    train = tf.placeholder(
        name = 'train',
        dtype = tf.bool,
        shape = ())
    mask = tf.placeholder(
        name = 'mask',
        dtype = tf.float32,
        shape = [len(Joint)]
    )

In [7]:
# input size 256 * 256 * 3

with tf.variable_scope('compress'):
    with tf.variable_scope('conv_bn_relu'):
        net = layer.conv(input = images, ksize = 7, kchannel = 64, kstride = 2) # 128 * 128 * 64
        net = layer.bn(input = net, train = train)
        net = layer.relu(input = net)

    net = module.bottleneck(input = net, kchannel = 128, train = train, name = 'A') # 128 * 128 * 128
    net = layer.pool(input = net) # 64 * 64 * 128
    net = module.bottleneck(input = net, kchannel = 128, train = train, name = 'B') # 64 * 64 * 128
    net = module.bottleneck(input = net, kchannel = 256, train = train, name = 'C') # 64 * 64 * 256

In [8]:
class tf_Spectrum:
    Color = tf.constant([
        [0, 0, 128],
        [0, 0, 255],
        [0, 255, 0],
        [255, 255, 0],
        [255, 0, 0]
    ], dtype = tf.float32)

def tf_gray2color(gray, spectrum = tf_Spectrum.Color):
    indices = tf.floor_div(gray, 64)
    
    t = tf.expand_dims((gray - indices * 64) / (64), axis = -1)
    indices = tf.cast(indices, dtype = tf.int32)
    
    return tf.add(
        tf.multiply(tf.gather(spectrum, indices), 1 - t),
        tf.multiply(tf.gather(spectrum, indices+1), t)
    )

def tf_merge(rgb, heat):
    heat = tf.image.resize_images(
        heat,
        [256, 256]
    )
    heat = tf.reduce_max(
        heat,
        axis = -1
    )
    return tf.cast(
        tf.add(
            tf.multiply(
                tf_gray2color(heat),
                0.6
            ),
            tf.multiply(rgb, 0.4)
        ),
        dtype = tf.uint8
    )

In [9]:
last_stage = 8
heatmaps = []
if FLAGS.task == 'eval':
    output = []

for stage in range(1, last_stage+1):
    with tf.variable_scope('hourglass_' + str(stage)):
        prev = tf.identity(net)
        net = module.hourglass(input = net, train = train) # 64 * 64 * 256

        with tf.variable_scope('inter_hourglasses'):
            net = module.bottleneck(input = net, train = train) # 64 * 64 * 256
            net = layer.conv(input = net, ksize = 1, kchannel = 256) # 64 * 64 * 256
            net = layer.bn(input = net, train = train)
            net = layer.relu(input = net)

        with tf.variable_scope('heatmap'):
            heatmap = layer.conv(input = net, ksize = 1, kchannel = len(Joint)) # 64 * 64 * joint
            
            heatmaps.append(heatmap)
            if FLAGS.task == 'eval':
                output.append(tf_merge(images, heatmap))

        if stage != last_stage:
            net = layer.conv(input = net, ksize = 1, kchannel = 256, name = 'inter')\
                + layer.conv(input = heatmap, ksize = 1, kchannel = 256, name = 'heatmap')\
                + prev # 64 * 64 * 256

In [10]:
if FLAGS.task == 'train' :
    with tf.variable_scope('loss'):
        loss = tf.losses.mean_squared_error(heatmaps_groundtruth, heatmaps[0] * mask)
        for stage in range(1, last_stage):
            loss = loss + tf.losses.mean_squared_error(heatmaps_groundtruth, heatmaps[stage] * mask)
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(name = 'optimizer', learning_rate = 0.00025).minimize(loss)

In [ ]:
sess = tf.Session()
saver = tf.train.Saver()

reader = DataCenter(root = FLAGS.path_to_data).request(data = FLAGS.name_of_data, task = FLAGS.task)

if FLAGS.load_pretrained:
    list_of_files = glob.glob(os.path.join(FLAGS.path_to_pretrained, '*'))
    latest_file = max(list_of_files, key=os.path.getctime)
    file_name = os.path.basename(latest_file).split('.ckpt')[0]
    saver.restore(sess, os.path.join(FLAGS.path_to_pretrained, file_name + '.ckpt'))
else:
    file_name = 'None'
    sess.run(tf.global_variables_initializer())

Check if the archive file is extracted...success!
	image path: /home/nulledge/Datasets/MPII/images
	annotation path: /home/nulledge/Datasets/MPII/mpii_human_pose_v1_u12_2
Load the annotation file...success!
Check if the index file is exists...success!
	train path: /home/nulledge/Datasets/MPII/train.txt
	eval path: /home/nulledge/Datasets/MPII/eval.txt


In [ ]:
if FLAGS.task == 'train':
    for epoch in range(1, FLAGS.epoch + 1):
        if FLAGS.name_of_data == 'FLIC':
            one_epoch = int(FLIC.NUMBER_OF_DATA * FLIC.TRAIN_RATIO)
        elif FLAGS.name_of_data == 'MPII':
            one_epoch = int((17266 + 1919) * MPII.TRAIN_RATIO)
        train_iter = tqdm(total = one_epoch, desc = 'epoch: ' + str(epoch) + '/' + str(FLAGS.epoch))
        reader.resetBatch()
        for i in range(one_epoch):
            train_images, train_heatmaps, _, train_mask = reader.getBatch(8)
            if train_images.shape[0] == 0:
                break
            _, result = sess.run([optimizer, loss],
                feed_dict = {
                    images: train_images,
                    heatmaps_groundtruth: train_heatmaps,
                    train: True,
                    mask: train_mask
                })
            train_iter.set_postfix(loss = result)
            train_iter.update(train_images.shape[0])
        train_iter.close();
        
        if FLAGS.save_pretrained:
            temp = saver.save(sess, os.path.join(FLAGS.path_to_pretrained, str(datetime.datetime.now()) + '_epoch' + str(epoch) + '.ckpt'))
else:
    if FLAGS.name_of_data == 'FLIC':
        one_epoch = FLIC.NUMBER_OF_DATA - int(FLIC.NUMBER_OF_DATA * FLIC.TRAIN_RATIO)
    elif FLAGS.name_of_data == 'MPII':
        one_epoch = (17266 + 1919) - int((17266 + 1919) * MPII.TRAIN_RATIO)
        
    reader.resetBatch()
    eval_iter = tqdm(total = one_epoch, desc = 'ckpt: ' + file_name)
    cnt = [0] * len(Joint)
    for i in range(one_epoch):
        eval_images, eval_heatmaps, eval_extra, eval_mask = reader.getBatch(8)
        if eval_images.shape[0] == 0:
            break
        to_cal, result = sess.run([heatmaps, output],
            feed_dict = {
                images: eval_images,
                heatmaps_groundtruth: eval_heatmaps,
                train: False})
        eval_iter.update(eval_images.shape[0])
        
        for batch in range(eval_images.shape[0]):
            for joint in Joint:
                if FLAGS.name_of_data == 'FLIC' and joint in FLIC.joint2index:
                    maximum = -1.0
                    px = -1
                    py = -1
                    for y in range(64):
                        for x in range(64):
                            if to_cal[-1][batch][y, x, joint.value - 1] > maximum:
                                maximum = to_cal[-1][batch][y, x, joint.value - 1]
                                py = y
                                px = x
                    dist = np.linalg.norm(
                        np.array([py, px])
                        - np.array(eval_extra[batch][0][joint.value - 1]))
                    if dist <= eval_extra[batch][1] * FLAGS.metric_coefficient:
                        cnt[joint.value - 1] += 1
                elif FLAGS.name_of_data == 'MPII' and joint in MPII.joint2index:
                    pass
            # imageio.imwrite('img/result' + str(i) + '_' + str(batch) + '_' + str(cnt) + '_' + str(len(FLIC.joint2index)) + '.png', result[-1][batch])
    for joint in Joint:
        if joint not in FLIC.joint2index:
            continue
        print(joint, cnt[joint.value - 1] / one_epoch)
                
            
    eval_iter.close()

epoch: 1/20:   0%|          | 0/17266 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:201: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:239: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:240: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
epoch: 2/20:  23%|██▎       | 3944/17266 [07:22<23:12,  9.57it/s, loss=84.3]

reader.resetBatch()
_, heat, _, _ = reader.getBatch(1)
heat = heat[0]
maximum = -1
for y in range(64):
    for x in range(64):
        heat[y, x, 0] = max(heat[y, x, :])
        if heat[y, x, 0] > maximum:
            maximum = heat[y, x, 0]
imageio.imwrite('img/heat.jpg', heat[:, :, 0])
print(maximum)